In [1]:
print("hello")

hello


In [2]:
# @title Pip install comands
!pip install librosa
!pip install torch torchvision torchaudio
!pip install torch
!pip install jiwer
!pip install wandb
!pip install datasets
!pip install kaggle

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple


In [3]:
#!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification  --path .
#!unzip gtzan-dataset-music-genre-classification.zip
#!rm unzip gtzan-dataset-music-genre-classification.zip

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
100%|█████████████████████████████████████▉| 1.21G/1.21G [00:45<00:00, 24.0MB/s]
100%|██████████████████████████████████████| 1.21G/1.21G [00:45<00:00, 28.6MB/s]
Archive:  gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inf

In [6]:
import os
import torchaudio
import torch
from torch.utils.data import DataLoader, Dataset

# Custom dataset class
class GTZANDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_paths = []
        self.labels = []
        self.genres = os.listdir(root_dir)
        
        for idx, genre in enumerate(self.genres):
            genre_dir = os.path.join(self.root_dir, genre)
            for file_name in os.listdir(genre_dir):
                file_path = os.path.join(genre_dir, file_name)
                self.file_paths.append(file_path)
                self.labels.append(idx)  # Use the genre index as the label

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        waveform, sample_rate = torchaudio.load(file_path)
        label = torch.tensor(self.labels[idx])

        # Apply any transformation if provided
        if self.transform:
            waveform = self.transform(waveform)
        
        return waveform, label

# Collate function for padding
def collate_fn(batch):
    waveforms = [item[0].squeeze(0) for item in batch] 
    labels = [item[1] for item in batch]
    
    waveforms = torch.nn.utils.rnn.pad_sequence(waveforms, batch_first=True)
    labels = torch.tensor(labels)
    
    return waveforms, labels



In [7]:
dataset = GTZANDataset(root_dir="data/genres_original/")
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

# Testing the DataLoader
for batch in dataloader:
    waveforms, labels = batch
    print(f"Waveforms shape: {waveforms.shape}, Labels: {labels}")
    break

Waveforms shape: torch.Size([16, 661794]), Labels: tensor([5, 4, 2, 5, 3, 6, 4, 0, 5, 1, 0, 8, 1, 7, 4, 1])
